In [80]:
import pandas as pd
import numpy as np

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score, accuracy_score

In [82]:
# 배깅 베이스모델
#base_model = DecisionTreeClassifier()
base_model = RandomForestClassifier()
#base_model = XGBClassifier()

# 배깅 모델 생성
bagging_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)

In [83]:
data = pd.read_csv('./dataset/final_naver.csv')
data

,Date,Open,High,Low,Close,Volume,Change,5MA,5Disparity,20MA,...,Short_left,Cos_sim,wil,buy,Fluctuation,label,C_O,H_L,H_C,C_L
0,2019-03-04,70300,71400,69900,70100,4931703,0.001429,72880.0,0.961855,74760.0,...,1856147,0,69050.0,0,0,0.000000,-200,1500,1300,200
1,2019-03-05,68300,70000,68300,69700,3465521,-0.005706,71740.0,0.971564,74450.0,...,1659421,0,69050.0,1,0,0.000000,1400,1700,300,1400
2,2019-03-06,68500,69600,67400,68100,4958303,-0.022956,70320.0,0.968430,74160.0,...,1337487,1,69350.0,1,0,0.000000,-400,2200,1500,700
3,2019-03-07,67200,68900,67100,68100,3402570,0.000000,69200.0,0.984104,73770.0,...,1160003,0,68300.0,1,0,0.000000,900,1800,800,1000
4,2019-03-08,67900,68200,66200,66700,3571792,-0.020558,68540.0,0.973154,73270.0,...,1188425,0,68800.0,0,0,0.000000,-1200,2000,1500,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,2024-02-23,166900,166900,159800,161400,7012032,0.031310,153560.0,1.051055,143035.0,...,1572988,0,168850.0,0,1,0.111111,-5500,7100,5500,1600
1231,2024-02-26,158400,164800,158400,161800,4329477,0.002478,155660.0,1.039445,144255.0,...,1427733,0,161950.0,1,0,0.111111,3400,6400,3000,3400
1232,2024-02-27,159000,160000,153300,153800,6107954,-0.049444,156500.0,0.982748,145145.0,...,1422572,0,162200.0,0,1,0.111111,-5200,6700,6200,500
1233,2024-02-28,154500,159200,153800,158000,3666960,0.027308,158300.0,0.998105,146295.0,...,1469505,1,157850.0,1,0,0.111111,3500,5400,1200,4200


In [84]:
# rise_fall 열 추가
data["rise_fall"] = 0
data.loc[data["Change"].shift(-1) > 0, "rise_fall"] = 1
data

In [203]:
df = data.copy()

In [86]:
#df.keys()

In [205]:
# 종속 변수와 독립 변수 분리
X = df.drop(['Date','rise_fall'], axis=1)
y = df['rise_fall']

In [206]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [207]:
# 배깅 모델을 학습
bagging_model.fit(X_train, y_train)

BaggingClassifier(estimator=RandomForestClassifier(), random_state=42)

In [208]:
# 예측
predictions = bagging_model.predict(X_test)
print(predictions)

[0 0 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1
 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 1
 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 1 0 1 1
 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0]


In [209]:
# 평가
#accuracy = accuracy_score(y_test, predictions)
#precision = precision_score(y_test, predictions)
precision = precision_score(y_test, predictions > 0.5)
recall = recall_score(y_test, predictions > 0.5)
f1 = f1_score(y_test, predictions > 0.5)

#print(f"Accuracy:, {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Precision: 0.531
Recall: 0.50
F1-Score: 0.51
